<a href="https://colab.research.google.com/github/Brianmax/ComputacionParalela/blob/main/Computacion_Paralela_(CUDA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-b0gubtq6
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-b0gubtq6


In [47]:
%load_ext nvcc_plugin


The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [48]:
%%cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cuda.h>
#include <math.h>
#include <iostream>
using namespace std;
__global__ void
matrixAddKernel_B(int *A, int *B, int *C, int TamMatrix) {
    int i = threadIdx.x + blockIdx.x*blockDim.x;
    int j = threadIdx.y + blockIdx.y*blockDim.y;
    if (i < TamMatrix && j < TamMatrix)
    {
        C[i*TamMatrix + j] = A[i*TamMatrix + j] + B[i*TamMatrix + j];
    }
}
__global__ void
matrixAddKernel_C(int *A, int *B, int *C, int TamMatrix) {
    int i = threadIdx.x + blockIdx.x*blockDim.x;
    int j = threadIdx.y + blockIdx.y*blockDim.y;
    if (j == 0 && i < TamMatrix) {
		while (j < TamMatrix) {
			C[i*TamMatrix + j] = A[i*TamMatrix + j] + B[i*TamMatrix + j];
			j++;
		}
	}
}
__global__ void
matrixAddKernel_D(int *A, int *B, int *C, int TamMatrix) {
    int i = threadIdx.x + blockIdx.x*blockDim.x;
    int j = threadIdx.y + blockIdx.y*blockDim.y;
    if (i == 0 && j < TamMatrix) {
		while (i < TamMatrix) {
			C[i*TamMatrix + j] = A[i*TamMatrix + j] + B[i*TamMatrix + j];
			i++;
		}
	}
}
void matrixAdd(int *h_A, int *h_B, int *h_C, int TamMatrix) {
    int *d_A, *d_B, *d_C;
    int size = TamMatrix*TamMatrix*sizeof(float);
    cudaMalloc((void**)&d_A, size);
    cudaMalloc((void**)&d_B, size);
    cudaMalloc((void**)&d_C, size);

    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_C, h_C, size, cudaMemcpyHostToDevice);
    dim3 dimGrid(ceil(TamMatrix / 32.0), ceil(TamMatrix / 32.0), 1);
	dim3 dimBlock(32.0, 32.0, 1);
    matrixAddKernel_C<<<dimGrid, dimBlock>>>(d_A, d_B, d_C, TamMatrix);

    
    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);

    cudaFree(d_A); cudaFree(d_B); cudaFree(d_C);
}

int main()
{
    int A[9] = {1,2,3,4,5,6,7,8,9};
    int B[9] = {1,2,3,4,5,6,7,8,9};
    int C[9];
    matrixAdd(A, B, C, 3);
    for (int i = 0; i < 9; i++)
    {
        cout << C[i] << " ";
    }
    return 0;
}


2 4 6 8 10 12 14 16 18 


In [45]:
%%cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cuda.h>
#include <math.h>
#include <iostream>
using namespace std;
__global__ 
void matrixMultKernel(int *A, int *B, int *C, int TamMatrix) {
    int i = threadIdx.x + blockIdx.x*blockDim.x;
    int j = threadIdx.y + blockIdx.y*blockDim.y;
    if (i < TamMatrix && j < TamMatrix)
    {
        int sum = 0;
        for (int it = 0; it < TamMatrix; it++)
            sum = sum + A[i*TamMatrix + it] * B[it];
        C[i] = sum;
    }
}
void matrixMult(int *h_A, int *h_B, int *h_C, int TamMatrix) {
    int *d_A, *d_B, *d_C;
    int size = TamMatrix*TamMatrix*sizeof(float);
    cudaMalloc((void**)&d_A, size);
    cudaMalloc((void**)&d_B, size);
    cudaMalloc((void**)&d_C, size);

    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_C, h_C, size, cudaMemcpyHostToDevice);
    dim3 dimGrid(ceil(TamMatrix / 32.0), ceil(TamMatrix / 32.0), 1);
	dim3 dimBlock(32.0, 32.0, 1);

    matrixMultKernel<<<dimGrid, dimBlock>>>(d_A, d_B, d_C, TamMatrix);

    
    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);

    cudaFree(d_A); cudaFree(d_B); cudaFree(d_C);
}

int main()
{
    int A[9] = {1,2,3,4,5,6,7,8,9};
    int B[9] = {1,2,3};
    int C[9];
    matrixMult(A, B, C, 3);
    for (int i = 0; i < 3; i++)
    {
        cout << C[i] << " ";
    }
    return 0;
}


14 32 50 
